In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn import ensemble, metrics, model_selection, naive_bayes
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from keras.layers import GlobalAveragePooling1D,Merge,Lambda,Input,GlobalMaxPooling1D
from catboost import CatBoostClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import warnings
warnings.filterwarnings('ignore')

/root/ml/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
X_train = pd.read_csv('train_huge_wv.csv')
X_test = pd.read_csv('test_huge_wv.csv')

In [3]:
df = pd.read_csv('10-15.csv')
df_test = pd.read_csv('10-15_test.csv')

In [6]:
columnns = list(set(list(X_train.columns) + list(df.columns)))

df_merge = df[[e for e in df.columns if e not in X_train.columns] + 'ID']
df_test_merge = [[e for e in df_test.columns if e not in X_test.columns] + 'ID']

['sent_vec_Внешний осмотр_220',
 'sent_vec_Общее состояние_230',
 'sent_vec_Диагноз_37',
 'sent_vec_Общее состояние_146',
 'sent_vec_Анамнез заболевания_30',
 'sent_vec_Диагноз_121',
 'sent_vec_Диагноз_218',
 'sent_vec_Внешний осмотр_154',
 'sent_vec_Анамнез заболевания_141',
 'same_Внешний осмотр_2',
 'sent_vec_Общее состояние_54',
 'same_аллергия_7',
 'sent_vec_Диагноз_31',
 'sent_vec_Анамнез заболевания_99',
 'same_аллергия_29',
 'sent_vec_Диагноз_280',
 'sent_vec_Диагноз_128',
 'sent_vec_Общее состояние_131',
 'sent_vec_Диагноз_69',
 'sent_vec_Внешний осмотр_73',
 'sent_vec_Общее состояние_195',
 'sent_vec_аллергия_181',
 'sent_vec_Общее состояние_241',
 'sent_vec_Диагноз_202',
 'same_Анамнез заболевания_175',
 'count_chars_nb_eap_аллергия',
 'sent_vec_Анамнез заболевания_84',
 'sent_vec_Анамнез заболевания_59',
 'sent_vec_аллергия_154',
 'sent_vec_Анамнез заболевания_149',
 'same_Анамнез заболевания_90',
 'sent_vec_Внешний осмотр_135',
 'sent_vec_Внешний осмотр_17',
 'sent_vec_Диа

In [9]:
X_train = X_train.merge(df[[e for e in df.columns if e not in X_train.columns] + ['ID']], on='ID')

In [11]:
X_test = X_test.merge(df_test[[e for e in df_test.columns if e not in X_test.columns] + ['ID']], on='ID')

In [14]:
X_test.to_csv('biggest_test.csv')

In [15]:
X_train.to_csv('biggest_train.csv')

In [3]:
X_train_new = X_train[[x for x in X_train.columns if x not in ['Revisit', 'Диагноз', 'Общее состояние', 'аллергия', 'Анамнез заболевания', 'Внешний осмотр']]]
X_test_new = X_test[[x for x in X_test.columns if x not in ['Диагноз', 'Общее состояние', 'аллергия', 'Анамнез заболевания', 'Внешний осмотр']]]

In [4]:
X_train_new = X_train_new.fillna(0.)
X_test_new = X_test_new.fillna(0.)

In [5]:
X_test_new.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
X_train_new.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [6]:
X_train_new = X_train_new.set_index('ID')
X_test_new = X_test_new.set_index('ID')

In [65]:
xttrain, xttest, yttrain, yttest = train_test_split(X_train_new, X_train['Revisit'], test_size=0.1, random_state=13)

In [66]:
xttrain_dl, xttest_dl, yttrain_dl, yttest_dl = train_test_split(xttrain, yttrain, test_size=0.2, random_state=13)

In [67]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xttrain_dl)
xvalid_glove_scl = scl.transform(xttest_dl)

xttest_scl = scl.transform(xttest)

In [87]:
xtrain_glove_scl.shape

(203821, 1525)

In [104]:
ytrain_enc = np_utils.to_categorical(yttrain_dl)
yvalid_enc = np_utils.to_categorical(yttest_dl)

In [111]:
model = Sequential()

model.add(Dense(128, input_dim=1525, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
# model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [112]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=3, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 203821 samples, validate on 50956 samples
Epoch 1/3
203821/203821 [==============================] - 49s 243us/step - loss: 0.4403 - acc: 0.8200 - val_loss: 0.4171 - val_acc: 0.8310
Epoch 2/3
203821/203821 [==============================] - 44s 214us/step - loss: 0.4225 - acc: 0.8278 - val_loss: 0.4163 - val_acc: 0.8310
Epoch 3/3
203821/203821 [==============================] - 47s 230us/step - loss: 0.4209 - acc: 0.8278 - val_loss: 0.4166 - val_acc: 0.8313


In [107]:
# np.unique(model.predict(xttest_scl))

array([0.00206846, 0.00211025, 0.00229271, ..., 0.99770725, 0.9978898 ,
       0.9979316 ], dtype=float32)

In [113]:
metrics.roc_auc_score(yttest, model.predict(xttest_scl)[:, 1])

0.7048000438401314

# GENERAL DL ANALYSIS

In [127]:
X_test_data = X_test[['ID', 'Код_диагноза', 'Диагноз',
       'Возраст', 'Пол', 'Общее состояние', 'аллергия', 'Анамнез заболевания',
       'Внешний осмотр', 'tfidf_nb_eap_tfidfAnamnes',
       'tfidf_nb_eap_tfidfDiagnoz', 'tfidf_nb_eap_tfidfGeneral',
       'tfidf_nb_eap_tfidfAllergy', 'tfidf_nb_eap_tfidfOutside',
       'tfidf_char_nb_eap_Диагноз', 'tfidf_char_nb_eap_Общее состояние',
       'tfidf_char_nb_eap_аллергия', 'tfidf_char_nb_eap_Анамнез заболевания',
       'tfidf_char_nb_eap_Внешний осмотр', 'count_words_nb_eap_Диагноз',
       'count_words_nb_eap_Общее состояние', 'count_words_nb_eap_аллергия',
       'count_words_nb_eap_Анамнез заболевания',
       'count_words_nb_eap_Внешний осмотр', 'Код_диагноза_0', 'Код_диагноза_1',
       'count_chars_nb_eap_Диагноз', 'count_chars_nb_eap_Общее состояние',
       'count_chars_nb_eap_аллергия', 'count_chars_nb_eap_Анамнез заболевания',
       'count_chars_nb_eap_Внешний осмотр']]

In [132]:
X_train_data = X_train[X_test_data.columns.tolist() + ['Revisit']]

In [133]:
X_train_glove = X_train[[c for c in X_train.columns.tolist() if c not in X_train_data.columns.tolist()] + ['ID', 'Revisit']]
X_test_glove = X_test[[c for c in X_test.columns.tolist() if c not in X_test_data.columns.tolist()] + ['ID']]

In [135]:
X_train_glove.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)
X_test_glove.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)

In [140]:
X_train_glove.head(5)

,sent_vec_Диагноз_0,sent_vec_Диагноз_1,sent_vec_Диагноз_2,sent_vec_Диагноз_3,sent_vec_Диагноз_4,sent_vec_Диагноз_5,sent_vec_Диагноз_6,sent_vec_Диагноз_7,sent_vec_Диагноз_8,sent_vec_Диагноз_9,...,sent_vec_Внешний осмотр_292,sent_vec_Внешний осмотр_293,sent_vec_Внешний осмотр_294,sent_vec_Внешний осмотр_295,sent_vec_Внешний осмотр_296,sent_vec_Внешний осмотр_297,sent_vec_Внешний осмотр_298,sent_vec_Внешний осмотр_299,ID,Revisit
0,0.001915,0.071876,-0.073892,0.147381,0.046674,-0.040121,-0.034678,0.066130,-0.106251,0.009980,...,-0.066182,-0.049977,-0.029365,-0.031125,-0.047419,0.021824,-0.072629,0.018202,0,1
1,-0.013145,0.048847,-0.046666,0.133570,-0.004592,-0.020951,0.008036,-0.001435,0.015957,0.053439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
2,0.037104,0.018268,-0.001514,0.119169,0.053053,-0.025556,0.004165,0.004449,0.010459,0.039660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1
4,-0.011424,-0.026490,0.017710,0.060962,0.017760,-0.032576,0.058018,-0.000649,0.010127,0.016562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1


In [141]:
earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

# NN
def doAddNN(X_train, X_test, pred_train, pred_test):
    X_train["nn_eap"] = pred_train[:,0]
    X_train["nn_hpl"] = pred_train[:,1]
    X_test["nn_eap"] = pred_test[:,0]
    X_test["nn_hpl"] = pred_test[:,1]
    return X_train,X_test

def initNN(nb_words_cnt,max_len):
    model = Sequential()
    model.add(Embedding(nb_words_cnt,32,input_length=max_len))
    model.add(Dropout(0.3))
    model.add(Conv1D(64,
                     5,
                     padding='valid',
                     activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(800, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return model

def doNN(X_train,X_test,Y_train):
    max_len = 70
    nb_words = 10000
    
    print('Processing text dataset')
    texts_1 = []
    for text in X_train['text']:
        texts_1.append(text)

    print('Found %s texts.' % len(texts_1))
    test_texts_1 = []
    for text in X_test['text']:
        test_texts_1.append(text)
    print('Found %s texts.' % len(test_texts_1))
    
    tokenizer = Tokenizer(num_words=nb_words)
    tokenizer.fit_on_texts(texts_1 + test_texts_1)
    sequences_1 = tokenizer.texts_to_sequences(texts_1)
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)

    xtrain_pad = pad_sequences(sequences_1, maxlen=max_len)
    xtest_pad = pad_sequences(test_sequences_1, maxlen=max_len)
    del test_sequences_1
    del sequences_1
    nb_words_cnt = min(nb_words, len(word_index)) + 1

    # we need to binarize the labels for the neural net
    ytrain_enc = np_utils.to_categorical(Y_train)
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([xtrain_pad.shape[0], 3])
    for dev_index, val_index in kf.split(xtrain_pad):
        dev_X, val_X = xtrain_pad[dev_index], xtrain_pad[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initNN(nb_words_cnt,max_len)
        model.fit(dev_X, y=dev_y, batch_size=32, epochs=4, verbose=1,validation_data=(val_X, val_y),callbacks=[earlyStopping])
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(xtest_pad)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddNN(X_train,X_test,pred_train,pred_full_test/5)

## NN Glove

def doAddNN_glove(X_train,X_test,pred_train,pred_test):
    X_train["nn_glove_eap"] = pred_train[:,0]
    X_train["nn_glove_hpl"] = pred_train[:,1]
    X_test["nn_glove_eap"] = pred_test[:,0]
    X_test["nn_glove_hpl"] = pred_test[:,1]
    return X_train,X_test

def initNN_glove():
    # create a simple 3 layer sequential neural net
    model = Sequential()

    model.add(Dense(128, input_dim=100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())

    model.add(Dense(2))
    model.add(Activation('softmax'))

    # compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def doNN_glove(X_train,X_test,Y_train,xtrain_glove,xtest_glove):
    # scale the data before any neural net:
    scl = preprocessing.StandardScaler()
    ytrain_enc = np_utils.to_categorical(Y_train)
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    xtrain_glove = scl.fit_transform(xtrain_glove)
    xtest_glove = scl.fit_transform(xtest_glove)
    pred_train = np.zeros([xtrain_glove.shape[0], 3])
    
    for dev_index, val_index in kf.split(xtrain_glove):
        dev_X, val_X = xtrain_glove[dev_index], xtrain_glove[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initNN_glove()
        model.fit(dev_X, y=dev_y, batch_size=32, epochs=10, verbose=1,validation_data=(val_X, val_y),callbacks=[earlyStopping])
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(xtest_glove)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddNN_glove(X_train,X_test,pred_train,pred_full_test/5)

# Fast Text

def doAddFastText(X_train,X_test,pred_train,pred_test):
    X_train["ff_eap"] = pred_train[:,0]
    X_train["ff_hpl"] = pred_train[:,1]
    X_test["ff_eap"] = pred_test[:,0]
    X_test["ff_hpl"] = pred_test[:,1]
    return X_train,X_test


def initFastText(embedding_dims,input_dim):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

def preprocessFastText(text):
    text = text.replace("' ", " ' ")
    signs = set(',.:;"?!')
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

def create_docs(df, n_gram_max=2):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in df.text:
        doc = preprocessFastText(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

def doFastText(X_train,X_test,Y_train):
    min_count = 2

    docs = create_docs(X_train)
    tokenizer = Tokenizer(lower=False, filters='')
    tokenizer.fit_on_texts(docs)
    num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

    tokenizer = Tokenizer(num_words=num_words, lower=False, filters='')
    tokenizer.fit_on_texts(docs)
    docs = tokenizer.texts_to_sequences(docs)

    maxlen = 300

    docs = pad_sequences(sequences=docs, maxlen=maxlen)
    input_dim = np.max(docs) + 1
    embedding_dims = 20

    # we need to binarize the labels for the neural net
    ytrain_enc = np_utils.to_categorical(Y_train)

    docs_test = create_docs(X_test)
    docs_test = tokenizer.texts_to_sequences(docs_test)
    docs_test = pad_sequences(sequences=docs_test, maxlen=maxlen)
    xtrain_pad = docs
    xtest_pad = docs_test
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([xtrain_pad.shape[0], 3])
    for dev_index, val_index in kf.split(xtrain_pad):
        dev_X, val_X = xtrain_pad[dev_index], xtrain_pad[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initFastText(embedding_dims,input_dim)
        model.fit(dev_X, y=dev_y, batch_size=32, epochs=25, verbose=1,validation_data=(val_X, val_y),callbacks=[earlyStopping])
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(docs_test)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddFastText(X_train,X_test,pred_train,pred_full_test/5)


In [143]:
Y_train = X_train_data['Revisit']
X_train_data.drop(['Revisit'], axis=1, inplace=True)

In [145]:
X_train_data = X_train_data.fillna(0.)
X_test_data = X_test_data.fillna(0.)

In [147]:
X_train_glove = X_train_glove.fillna(0.)
X_test_glove = X_test_glove.fillna(0.)

In [148]:
X_train, X_test = doNN_glove(X_train_data, X_test_data, Y_train, X_train_glove, X_test_glove)

MemoryError: 

In [ ]:
X_train, X_test = doFastText(X_train,X_test,Y_train)

In [ ]:
X_train,X_test = doNN(X_train,X_test,Y_train)

In [2]:
X_train_glove

NameError: name 'X_train_glove' is not defined